In [2]:
%load_ext autoreload
%autoreload 2

In [31]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [4]:
# подгружаем .env
load_dotenv()
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [53]:
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

TABLE = 'flats_price'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn).drop(columns=['id'])

dst_conn.dispose()

#### Типы данных

In [54]:
features = data.drop(columns = ['flat_id', 'price'])
print(features.dtypes.value_counts())

int64      7
float64    6
bool       3
Name: count, dtype: int64


#### Перевод boolean в int 

In [1]:
# def bool_to_int(data: pd.DataFrame) -> pd.DataFrame:
#     bool_columns = data.select_dtypes(include=bool).columns.tolist()
#     for col in bool_columns:
#         data[col] = data[col].astype(int)
#     return data

# data = bool_to_int(data)
# data.describe()

#### Перевод ID из int в object

In [56]:
def int_to_str(data: pd.DataFrame) -> pd.DataFrame:
    columns = ['flat_id', 'building_id']
    for col in columns:
        data[col] = data[col].astype(str)
    return data

data = int_to_str(data)
data.describe()

,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
count,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,141362.0,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,141362.000000,1.413620e+05
mean,7.467346,0.009706,9.001579,31.056948,2.129476,0.0,62.374644,1986.600048,3.232941,55.730059,37.589235,2.753650,251.993230,14.107554,0.897384,1.944162e+07
std,5.717144,0.098038,5.264076,23.968640,0.994340,0.0,40.295864,22.136409,1.459461,0.102611,0.150122,0.223275,207.336169,6.898045,0.303458,6.626954e+07
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,11.000000,1901.000000,0.000000,55.211460,36.864372,2.000000,1.000000,1.000000,0.000000,1.100000e+01
25%,3.000000,0.000000,6.100000,19.000000,1.000000,0.0,39.299999,1969.000000,2.000000,55.653858,37.491764,2.640000,111.000000,9.000000,1.000000,8.900000e+06
50%,6.000000,0.000000,8.800000,29.400000,2.000000,0.0,53.000000,1985.000000,4.000000,55.724686,37.581146,2.640000,200.000000,14.000000,1.000000,1.185000e+07
75%,10.000000,0.000000,10.200000,41.400002,3.000000,0.0,72.000000,2007.000000,4.000000,55.807323,37.691055,2.800000,324.000000,17.000000,1.000000,1.695000e+07
max,56.000000,1.000000,203.000000,700.000000,20.000000,0.0,960.299988,2023.000000,6.000000,56.011032,37.946411,27.000000,4455.000000,99.000000,1.000000,9.873738e+09


#### Дубли flat_id

In [57]:
is_duplicated_id = data.duplicated(subset=['flat_id'], keep=False)
print(sum(is_duplicated_id))

0


#### Дубли в признаках

In [58]:
feature_cols = data.drop(columns = ['flat_id', 'building_id', 'price']).columns.to_list()
is_duplicated_features = data[feature_cols].duplicated(keep=False)
print(len(data[is_duplicated_features]))
print(len(data[~is_duplicated_features]))
print(len(data))

36488
104874
141362


In [59]:
feature_cols = data.drop(columns = ['flat_id', 'building_id']).columns.to_list()
is_duplicated_features = data[feature_cols].duplicated(keep=False)
print(len(data[is_duplicated_features]))
print(len(data[~is_duplicated_features]))
print(len(data))

17425
123937
141362


In [60]:
# Удалить дубли, учитывая фичи и таргет. Оставить одно из значений.
def remove_features_target_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    feature_cols = data.drop(columns = ['flat_id', 'building_id']).columns.to_list()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep='first')
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

data = remove_features_target_duplicates(data)
data.describe()

,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
count,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,132246.0,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,132246.000000,1.322460e+05
mean,7.455651,0.009603,8.964356,30.990703,2.124200,0.0,62.038371,1986.492476,3.241240,55.730344,37.589355,2.752011,251.587723,14.077605,0.896866,1.933210e+07
std,5.703849,0.097525,5.212305,23.667590,0.990495,0.0,39.784039,22.067968,1.459392,0.102887,0.150453,0.222892,206.994097,6.881819,0.304134,6.757713e+07
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,11.000000,1901.000000,0.000000,55.211460,36.864372,2.000000,1.000000,1.000000,0.000000,1.100000e+01
25%,3.000000,0.000000,6.100000,19.000000,1.000000,0.0,39.200001,1969.000000,2.000000,55.653915,37.491764,2.640000,111.000000,9.000000,1.000000,8.900000e+06
50%,6.000000,0.000000,8.700000,29.400000,2.000000,0.0,52.900002,1985.000000,4.000000,55.724754,37.581272,2.640000,199.000000,14.000000,1.000000,1.180000e+07
75%,10.000000,0.000000,10.200000,41.000000,3.000000,0.0,71.699997,2007.000000,4.000000,55.808098,37.692499,2.800000,323.000000,17.000000,1.000000,1.685000e+07
max,56.000000,1.000000,203.000000,700.000000,20.000000,0.0,960.299988,2023.000000,6.000000,56.011032,37.946411,27.000000,4455.000000,99.000000,1.000000,9.873738e+09


In [61]:
# Удалить дубли, учитывая только фичи, так как при одинаковых фичах наблюдаются разные значения таргета. Не оставлять дуюлирующиейся значения.
def remove_features_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    feature_cols = data.drop(columns = ['flat_id', 'building_id', 'price']).columns.to_list()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

data = remove_features_duplicates(data)
data.describe()

,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
count,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.0,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,1.117870e+05
mean,7.476665,0.009545,8.931259,30.770191,2.113591,0.0,61.412430,1986.498206,3.250396,55.730673,37.588674,2.749234,251.532459,14.097587,0.896607,1.863220e+07
std,5.695009,0.097231,5.201291,23.360291,0.980887,0.0,38.402714,21.945652,1.459673,0.103575,0.150873,0.219750,205.799167,6.858261,0.304473,6.299405e+07
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,11.000000,1901.000000,0.000000,55.211460,36.864372,2.000000,1.000000,1.000000,0.000000,1.100000e+01
25%,3.000000,0.000000,6.100000,19.000000,1.000000,0.0,39.099998,1969.000000,2.000000,55.653801,37.491585,2.640000,111.000000,9.000000,1.000000,8.850000e+06
50%,6.000000,0.000000,8.700000,29.299999,2.000000,0.0,52.500000,1985.000000,4.000000,55.724754,37.581146,2.640000,200.000000,14.000000,1.000000,1.170000e+07
75%,10.000000,0.000000,10.200000,41.000000,3.000000,0.0,70.900002,2006.000000,4.000000,55.809029,37.691910,2.800000,323.000000,17.000000,1.000000,1.650000e+07
max,56.000000,1.000000,203.000000,700.000000,20.000000,0.0,960.299988,2023.000000,6.000000,56.011032,37.946411,27.000000,4455.000000,99.000000,1.000000,9.799999e+09


#### Заполнить пропущенные значения

In [62]:
def fill_missing_values(data: pd.DataFrame) -> pd.DataFrame:
	cols_with_nans = data.isnull().sum()
	cols_with_nans = cols_with_nans[cols_with_nans > 0]
	for col in cols_with_nans:
		if data[col].dtype in [float, int]:
			fill_value = data[col].mean()
		data[col] = data[col].fillna(fill_value)
	return data

data = fill_missing_values(data)
data.describe()

,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
count,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.0,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,111787.000000,1.117870e+05
mean,7.476665,0.009545,8.931259,30.770191,2.113591,0.0,61.412430,1986.498206,3.250396,55.730673,37.588674,2.749234,251.532459,14.097587,0.896607,1.863220e+07
std,5.695009,0.097231,5.201291,23.360291,0.980887,0.0,38.402714,21.945652,1.459673,0.103575,0.150873,0.219750,205.799167,6.858261,0.304473,6.299405e+07
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,11.000000,1901.000000,0.000000,55.211460,36.864372,2.000000,1.000000,1.000000,0.000000,1.100000e+01
25%,3.000000,0.000000,6.100000,19.000000,1.000000,0.0,39.099998,1969.000000,2.000000,55.653801,37.491585,2.640000,111.000000,9.000000,1.000000,8.850000e+06
50%,6.000000,0.000000,8.700000,29.299999,2.000000,0.0,52.500000,1985.000000,4.000000,55.724754,37.581146,2.640000,200.000000,14.000000,1.000000,1.170000e+07
75%,10.000000,0.000000,10.200000,41.000000,3.000000,0.0,70.900002,2006.000000,4.000000,55.809029,37.691910,2.800000,323.000000,17.000000,1.000000,1.650000e+07
max,56.000000,1.000000,203.000000,700.000000,20.000000,0.0,960.299988,2023.000000,6.000000,56.011032,37.946411,27.000000,4455.000000,99.000000,1.000000,9.799999e+09


#### Удалить выбросы

In [63]:
def remove_outliers(data: pd.DataFrame, threshold = 1.5) -> pd.DataFrame:
    columns = ['price', 'flats_count', 'ceiling_height', 'total_area', 'living_area', 'kitchen_area']
    potential_outliers = pd.DataFrame()

    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        margin = IQR * threshold
        lower = Q1 - margin
        upper = Q3 + margin
        potential_outliers[col] = ~data[col].between(lower, upper)

    outliers = potential_outliers.any(axis=1)
    return data[~outliers]

data = remove_outliers(data)
data.describe()

,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,price
count,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,90903.0,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,90903.000000,9.090300e+04
mean,7.039317,0.006160,8.110216,27.406252,1.939089,0.0,51.637527,1984.778093,3.512810,55.730410,37.595705,2.688830,224.964281,13.322366,0.894976,1.169100e+07
std,4.970607,0.078247,3.049814,14.420754,0.813611,0.0,16.182603,19.530225,1.408035,0.107745,0.158100,0.131417,134.665456,5.547405,0.306586,4.457981e+06
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,11.000000,1902.000000,0.000000,55.211460,36.864372,2.450000,1.000000,1.000000,0.000000,1.100000e+01
25%,3.000000,0.000000,6.000000,19.000000,1.000000,0.0,38.500000,1969.000000,2.000000,55.648724,37.491440,2.640000,112.000000,9.000000,1.000000,8.500000e+06
50%,6.000000,0.000000,8.500000,28.299999,2.000000,0.0,48.099998,1980.000000,4.000000,55.718708,37.588718,2.640000,196.000000,12.000000,1.000000,1.070000e+07
75%,10.000000,0.000000,10.000000,36.000000,3.000000,0.0,60.500000,2003.000000,4.000000,55.816494,37.719379,2.740000,306.000000,17.000000,1.000000,1.395000e+07
max,55.000000,1.000000,16.300000,74.000000,7.000000,0.0,118.500000,2023.000000,6.000000,56.011032,37.946411,3.000000,640.000000,99.000000,1.000000,2.795000e+07
